# Quarta lezione

In [ ]:
# This cell loads some html style files (it may be either run/ignored/deleted)
from IPython.core.display import HTML
with open( '../lezioni/style/custom.css', 'r' ) as f: html_style = f.read()
HTML( html_style )

Leggiamo la stessa tabella delle lezioni scorse. 

Leggo solo la colonna `soil` che contiene la temperatura media giornaliera (misurata nel terreno a 10cm di profondità).

In [ ]:
import pandas as pd
df = pd.read_csv('../dati/Cork_Airport.csv', 
                 skiprows=24,
                 usecols=['date', 'maxtp', 'mintp', 'soil'],
                )
df.head(2)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()
p = figure(x_axis_type = "datetime",
           width = 700, height = 250,
           tools = 'xwheel_zoom,reset, crosshair, xpan, xbox_zoom',
          )
show( p )